# Reading and Preprocessing

### Reading the data files

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime

# load the data
data_dir = os.getcwd() + '\\data\\'

raw_co2_1 = pd.read_csv(data_dir + 'co2_1k.csv', header=None)
raw_co2_2 = pd.read_csv(data_dir + 'co2_10k.csv', header=None)

raw_light_1 = pd.read_csv(data_dir + 'light0.csv', header=None)
raw_light_2 = pd.read_csv(data_dir + 'light1.csv', header=None)

raw_motion = pd.read_csv(data_dir + 'motion.csv', header=None)

raw_power_1 = pd.read_csv(data_dir + 'power0.csv', header=None)
raw_power_2 = pd.read_csv(data_dir + 'power1.csv', header=None)

raw_sound = pd.read_csv(data_dir + 'sound.csv')

raw_schedule = pd.read_csv(data_dir + 'schedule.csv')

## Preparing data

### Merging data
Here are the abbreviations of column names:
- ms: microseconds
- tav: time-averaged value
- tace: time-averaged compression error
- sc: number of raw samples aggregated

In [2]:
raw_co2_1.columns = ['ms', 'co2_1_tav', 'co2_1_tac', 'co2_1_sc'] 
raw_co2_2.columns = ['ms', 'co2_2_tav', 'co2_2_tac', 'co2_2_sc']  
raw_light_1.columns = ['ms', 'light_1_tav', 'light_1_tac', 'light_1_sc']   
raw_light_2.columns = ['ms', 'light_2_tav', 'light_2_tac', 'light_2_sc']   
raw_motion.columns = ['ms', 'motion_tav', 'motion_tac', 'motion_sc']   
raw_power_1.columns = ['ms', 'power_1_tav', 'power_1_tac', 'power_1_sc']   
raw_power_2.columns = ['ms', 'power_2_tav', 'power_2_tac', 'power_2_sc']   
raw_sound.columns = ['ms', 'sound_tav', 'sound_tac', 'sound_sc']   
raw_schedule.columns = ['ms', 'schedule_tav', 'schedule_tac', 'schedule_sc']   

### Captured time frame (in milliseconds) of each sensor 

In [3]:
print('CO2 1: ', raw_co2_1['ms'].min(), raw_co2_1['ms'].max())
print('CO2 2:', raw_co2_2['ms'].min(), raw_co2_2['ms'].max())
print('Light 1:', raw_light_1['ms'].min(), raw_light_1['ms'].max())
print('Light 2:', raw_light_2['ms'].min(), raw_light_2['ms'].max())
print('Motion:', raw_motion['ms'].min(), raw_motion['ms'].max())
print('Power 1:', raw_power_1['ms'].min(), raw_power_1['ms'].max())
print('Power 2:', raw_power_2['ms'].min(), raw_power_2['ms'].max())
print('Sound:', raw_sound['ms'].min(), raw_sound['ms'].max())
print('Schedule:', raw_schedule['ms'].min(), raw_schedule['ms'].max())

CO2 1:  1279237740000000 1280251200000000
CO2 2: 1279237740000000 1280252280000000
Light 1: 1279237740000000 1280252220000000
Light 2: 1279237740000000 1280251320000000
Motion: 1279237740000000 1280251200000000
Power 1: 1279237740000000 1280251320000000
Power 2: 1279237740000000 1280252940000000
Sound: 1279237800000000 1280252700000000
Schedule: 1279199760000000 1280250300000000


We can see that captured data for all sensors and ground truth is range [1279237800000000, 1280250300000000]. Therefore, we should remove the data outside of the range above.

In [4]:
min_time = 1279237800000000
max_time = 1280250300000000

# Filtering the datasets for the specific range
raw_co2_1 = raw_co2_1[(raw_co2_1.ms >= min_time) & (raw_co2_1.ms <= max_time)]
raw_co2_2 = raw_co2_2[(raw_co2_2.ms >= min_time) & (raw_co2_2.ms <= max_time)]
raw_light_1 = raw_light_1[(raw_light_1.ms >= min_time) & (raw_light_1.ms <= max_time)]
raw_light_2 = raw_light_2[(raw_light_2.ms >= min_time) & (raw_light_2.ms <= max_time)]
raw_motion = raw_motion[(raw_motion.ms >= min_time) & (raw_motion.ms <= max_time)]
raw_power_1 = raw_power_1[(raw_power_1.ms >= min_time) & (raw_power_1.ms <= max_time)]
raw_power_2 = raw_power_2[(raw_power_2.ms >= min_time) & (raw_power_2.ms <= max_time)]
raw_sound = raw_sound[(raw_sound.ms >= min_time) & (raw_sound.ms <= max_time)]
raw_schedule = raw_schedule[(raw_schedule.ms >= min_time) & (raw_schedule.ms <= max_time)]

In [5]:
# Converting millisecond column to index
raw_co2_1 = raw_co2_1.set_index(['ms'])
raw_co2_2 = raw_co2_2.set_index(['ms'])
raw_light_1 = raw_light_1.set_index(['ms'])
raw_light_2 = raw_light_2.set_index(['ms'])
raw_motion = raw_motion.set_index(['ms'])
raw_power_1 = raw_power_1.set_index(['ms'])
raw_power_2 = raw_power_2.set_index(['ms'])
raw_sound = raw_sound.set_index(['ms'])
raw_schedule = raw_schedule.set_index(['ms'])

# Merging all caputred data (outer join)
data = pd.concat([raw_co2_1, raw_co2_2, raw_light_1, raw_light_2, 
           raw_motion, raw_power_1, raw_power_2, raw_sound, raw_schedule], axis=1)

# Extracting date-time from milliseconds
data['time'] = pd.to_datetime(data.index / 1000, unit='ms')

In [6]:
# Number of NULL columns in data (due to above outer join)
data.isnull().sum().sum()

0

It shows that there is no NULL column data in the merged dataset.

### Saving cleansed data

In [7]:
data.to_csv(data_dir + '1.csv', index = True)